<a href="https://colab.research.google.com/github/ConSeanway/invSim/blob/master/Cross_Node_Implementation_Inventory_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inventory Simulation
## Authors:  Sean Conway + Yanzhe Ma

---


Last Modified: 31AUG2020

> Implementation of Inventory Simulation Using Classes



Inventory Simulation Class

In [79]:
import numpy as np
import pandas as pd
#from scipy.stats import norm


# "Game" class that can be created to run the whole simulation
class InvSimulation:
  def __init__(self, periodsToSimulate=1000):
    self.periodsToSimulate = periodsToSimulate

    # Contains all of the nodes in our simulation (reference by ID)
    self.nodeDict = {}

#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #

# Node - related methods

  def createNode(self, nodeID, h, p, nodeType, demandMean, demandStDev):
    '''
    Creates a node to be used in our supply network
    In:  InvSim object, nodeID, h, p, nodeType, demandMean, demandStDev
    '''
    self.nodeDict[nodeID] = Node(nodeID, h, p, nodeType,demandMean, demandStDev)
  
  #links together two nodes in preDict and recDict; could add a boolean later for linking both ways
  def linkNode(self,startNode,endNode):

    '''
    Create a unidirectional link between nodes
    In: InvSim object, starting Node index, ending Node index
    '''

    if startNode in self.nodeDict[startNode].recDict.keys():
      self.nodeDict[startNode].recDict[startNode].append(endNode)
    else:
      self.nodeDict[startNode].recDict[startNode] = []
      self.nodeDict[startNode].recDict[startNode].append(endNode)
    
    if endNode in self.nodeDict[endNode].preDict.keys():
      self.nodeDict[endNode].preDict[endNode].append(startNode)
    else:
      self.nodeDict[endNode].preDict[endNode] = []
      self.nodeDict[endNode].preDict[endNode].append(startNode)

#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #

  def playSimulation(self, gameType="multiNodeVerify", BSLevel=60):
    '''
    Play the simulation, given the following:
    - game type (string) (default="multiNodeVerify)
    - base stock level (integer for multiNodeVerify game), single value for all nodes (default=60)
    '''

    if gameType == "multiNodeVerify":
      self.multiNodeVerify(df["IO"], BSLevel)
    else:
      self.playOptimalBaseStockGame()

#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #
  def printEndStats(self, nodeID, thisNode, period):
      '''
      Print the resulting statistics for a node in the game
      In:
      - nodeID (integer)
      - thisNode (node Object)
      - period (integer)
      '''

      if period == self.periodsToSimulate - 1:
        print("(IL) Starting Inventory record for node " + str(nodeID) + ":" + str(thisNode.startingInventoryRecord))
        print("(IS) Inbound supply record for node " + str(nodeID) + ":" + str(thisNode.receivedMats))
        print("(IO) Demand for node " + str(nodeID) + ":" + str(thisNode.demandArray))
        print("(OQ) Order for node " + str(nodeID) + ":" + str(thisNode.orderArray))
        print("(DMFS) Supply for node " + str(nodeID) + ":" + str(thisNode.supplyArray))
        print("(EIL) Ending Inventory record for node " + str(nodeID) + ":" + str(thisNode.endingInventoryRecord))
        print("(BO) Backorders for node " + str(nodeID) + ":" + str(thisNode.backorderRecord))
        print("(TC) Total Cost for node " + str(nodeID) + ":" + str(thisNode.costRecord))
        print()

        #diffIS = []
        #if nodeID == 2:
        #  for i in range(0,min(len(thisNode.inBoundOrders),len(df["IS 3 Node"]))):
        #    diffIS.append(thisNode.inBoundOrders[i]-df["IS 3 Node"][i])
        #  print(diffIS)
        #  print(len(thisNode.inBoundOrders))

  def getReceivedMaterials(self, nodeID, thisNode, period):
    '''
    Get the number of inbound materials from what the preceding node was able to supply

    Example:
    Flow of material
    o -> o

    Suppose the left node represents a wholesaler, and the right node represents a retailer
    How much material was the wholesaler able to supply to the retailer?

    In:
    - nodeID (dictionary key)
    - period (integer)

    Out:
    - number of received materials (numeric)
    '''

    # Get the number of inbound materials from what the previous node was able to supply
    receivedMats = 0
    recipientNum = 0
    #recipient = []
    if nodeID < max(self.nodeDict.keys()) and period != 0:
      #recipient = self.nodeDict[nodeID].recDict[nodeID]
      #for i in range(0,len(recipient)):
        #recipientNum = self.nodeDict[recipient[i]].recDict[recipient[i]]
        #inBoundOrders = inBoundOrders + max(self.nodeDict[recipient[i]].supplyArray[period-1]/recipientNum,0)
        #----------------------------------
        #This part of the code above is not right, but could use as a reference
      receivedMats = max(self.nodeDict[nodeID + 1].supplyArray[period - 1], 0)
    elif nodeID == max(self.nodeDict.keys()) and period != 0:
      receivedMats = max(self.nodeDict[nodeID].orderArray[period - 1], 0)
    thisNode.receivedMats.append(receivedMats)
    return receivedMats

  def computeDemand(self, nodeID, thisNode, demandArray, period):
    '''
    Compute the demand for a given node (using a demand array as reference)
    In:
    - nodeID (dictionary Key)
    - thisNode (Node)
    - demandArray (array of numerics)
    - period (integer)
    '''

    # Pull demand from the demand array if it's the retailer
    # Upstream nodes look at what the previous node's order was (that is in turn their demand)
    if nodeID == 0:
      thisPeriodDemand = demandArray[period]
    else:
      #downstreamList = self.nodeDict[nodeID].recDict[nodeID]
      #for i in range(0,len(recipient)):
      #  downstreamNum = self.nodeDict[downstreamList[i]].recDict[downstreamList[i]]
      #  inBoundMats = inBoundMats + max(self.nodeDict[downstreamList[i]].supplyArray[period-1]/downstreamNum,0)
        #------------------------------------------------
        #------------------------------------------------
        #------------------------------------------------
      upStreamNode = self.nodeDict[nodeID - 1]
      thisPeriodDemand = upStreamNode.orderArray[period]

    # Incur the demand by appending it to the node's demand array (this is basically just being pulled from the file)
    thisNode.demandArray.append(thisPeriodDemand)
    return thisPeriodDemand

    
  def satisfyDemand(self, receivedMats, thisNode, thisPeriodDemand, backordersFulfilled):

    '''
    Given the demand, as well as the supply for a node for a current period, compute
    how much of this node's demand can be supplied (and how many backorders result)

    Record this information in the node

    In:
    - Received materials (numeric)
    - node object (Node)
    - demand for this period (numeric)
    - backorders fulfilled (numeric)

    '''

    availableSupply = receivedMats + thisNode.startingInventory

    # Record demand that can be supplied, along with the backorders that were fulfilled
    suppliableDemand = min(availableSupply, thisPeriodDemand)
    thisNode.supplyArray.append(suppliableDemand + backordersFulfilled)
    backorders = max(thisPeriodDemand - suppliableDemand, 0)
    thisNode.backorders = backorders
    thisNode.backorderRecord.append(backorders)

  def computeEIAndCosts(self, thisNode, thisPeriodDemand, receivedMats):
    thisNode.endingInventory = thisNode.startingInventory - thisPeriodDemand + receivedMats
    thisPeriodCost = max(0,thisNode.endingInventory*thisNode.holdingCost)+max(0,-1*thisNode.endingInventory*thisNode.stockoutCost)
    thisNode.endingInventoryRecord.append(thisNode.endingInventory)
    thisNode.costRecord.append(thisPeriodCost)

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #

  def multiNodeVerify(self, demandArray, BSLevel=60):

    '''
    Run instance of the game using a preset demand
    In:
    - demandArray:  array of numerics, containing demand for each node
    - BSLevel: (optional, default=60) Base stock level (numeric)
    '''

    # Reset lists from previous runs
    for nodeID in sorted(self.nodeDict.keys()): self.nodeDict[nodeID].initGame(BSLevel)

    # Determine order quantity for the current period for all nodes (prior to demand determination)
    for period in range(0, self.periodsToSimulate):
      for nodeID in sorted(self.nodeDict.keys()):

        thisNode = self.nodeDict[nodeID]       

        # Record starting inventory, and get materials from upstream node
        thisNode.startingInventoryRecord.append(thisNode.startingInventory)
        receivedMats = self.getReceivedMaterials(nodeID, thisNode, period)        

        # Determine how many backorders can be fulfilled
        backordersFulfilled, receivedMats = thisNode.getBackorders(receivedMats)

        # Compute the demand, and satisfy as much of it as we are able (during this period)
        thisPeriodDemand = self.computeDemand(nodeID, thisNode, demandArray, period)
        self.satisfyDemand(receivedMats, thisNode, thisPeriodDemand, backordersFulfilled)

        # Order the same demand that you had last period
        qtyToOrder = thisPeriodDemand
        thisNode.orderArray.append(qtyToOrder)

        # Compute the ending inventory and costs for this period (and print resulting statistics)
        self.computeEIAndCosts(thisNode, thisPeriodDemand, receivedMats)
        self.printEndStats(nodeID, thisNode, period)
        
        # Make the starting inventory equal to ending inventory from previous period
        thisNode.startingInventory = thisNode.endingInventory

In [80]:
class Node:

  def __init__(self, id, h=3, p=100, nodeType = "retailer",demandMean=50, demandStDev=10):

    '''
    Node represents a single node on our supply network
    In:
    - NodeID (required, we recommend using integers 0-inf)
    - h (unit holding cost) (numeric), default = 3
    - p (unit stockout cost) (numeric), default = 100
    - nodeType (description of node type) (string), default = "retailer"
    - demandMean (mean of the demand function) (numeric), default = 50
    - demandStDev (standard deviation of the demand function) (numeric), default = 10

    Note that we are currently assuming normal demands (perhaps specify other distributions if you want)
    '''

    self.id = id
    
    self.holdingCost = h
    self.stockoutCost = p
    self.baseStockLevel = 0

    self.startingInventory = self.baseStockLevel
    self.endingInventory = 0
    
    self.nodeType = nodeType
    self.backorders = 0

    # This is assuming that in this game, we have an idea of the distribution params for demand
    self.demandMean = demandMean
    self.demandStDev = demandStDev

    # Add 2 dictionaries, one for recording recipients and one for predecessors, to each node for cross-node implementations
    self.preDict = {}
    self.recDict = {}

  def initGame(self, BSLevel):
    self.demandArray  = []
    self.orderArray = []
    self.supplyArray = []
    
    self.startingInventoryRecord = []
    self.endingInventoryRecord = []
    self.backorderRecord = []
    self.receivedMats = []
    self.costRecord = []
    # Initialize the base stock level for period 0 and beyond, also starting inventory for period 0
    self.startingInventory = BSLevel
    self.baseStockLevel = BSLevel

  def getBackorders(self, receivedMats):
    backordersFulfilled = 0

    thisNode = self

    # Serve backorders with the new supply first
    if thisNode.backorders > 0:
      if receivedMats >= thisNode.backorders:
        backordersFulfilled = thisNode.backorders
        thisNode.backorders = 0
        receivedMats = receivedMats - thisNode.backorders
      else:
        thisNode.backorders = thisNode.backorders - receivedMats
        receivedMats = 0
    return backordersFulfilled, receivedMats

In [77]:
df = pd.read_csv("3_node_60_60_60.csv")
print(df["IS 3 Node"][len(df["IS 3 Node"])-1])


44.49149011


In [81]:

myInvSim = InvSimulation()

# Node creation:  Key (mandatory), holding cost, stockout cost, and fixed order cost
myInvSim.createNode(nodeID = 0, h = 10, p = 100, nodeType = "retailer", demandMean=50, demandStDev=10)
myInvSim.createNode(nodeID = 1, h = 10, p = 20, nodeType = "retailer", demandMean=50, demandStDev=10)
myInvSim.createNode(nodeID = 2, h = 10, p = 20, nodeType = "retailer", demandMean=50, demandStDev=10)
myInvSim.createNode(nodeID = 3, h = 10, p = 0, nodeType = "retailer", demandMean=50, demandStDev=10)

# Node linkage:  start Node Key, end Node Key
myInvSim.linkNode(startNode = 0,endNode = 1)
myInvSim.linkNode(startNode = 0,endNode = 2)
myInvSim.linkNode(startNode = 1,endNode = 3)
myInvSim.linkNode(startNode = 2,endNode = 3)
#print(myInvSim.nodeDict[0].recDict[0])
#print(myInvSim.nodeDict[2].preDict[2])

# Currently assume that everyone plays with the same policy
myInvSim.playSimulation(gameType = "multiNodeVerify", BSLevel=60)


(IL) Starting Inventory record for node 0:[60, 19.176249730000002, 7.852610339999998, -19.19040905, -21.04642566999999, 2.252469910000009, 8.468629170000007, 1.1208494700000102, 17.29876971000001, 14.300057680000009, 8.831042190000005, 16.512934380000004, -1.384173969999992, 0.15576768000001806, 1.8637772600000062, 23.540799350000007, -1.7492203399999937, 30.684211230000013, 23.496188270000005, -0.36212937999999184, -7.002773519999998, 1.1357251300000115, 20.347975239999997, -1.9206440600000008, -6.4190656899999965, -18.583998109999996, -9.13245520999999, -8.43921250999999, -14.179310299999997, -18.771425939999986, -9.480367459999997, 2.080773690000001, 8.348442799999987, 20.95334325999999, -0.36735494000000557, -1.9418782799999974, 26.49598647, 7.0024714999999915, 14.033030059999994, 9.32133035999999, 22.940767519999994, 2.725434529999994, 32.19033244999999, 1.3268761699999914, 32.492687939999996, 17.007861189999996, 9.516744180000003, 2.9750997399999974, 0.22282443999999657, -1.57310